## Graph Attention Network - Keras (Node Classification)

Graph Neural Network에서는 여러 Task를 수행합니다. 대표적인 2가지의 Task는 아래와 같습니다.
1. Node Classification: 해당 Node가 어떠한 Class에 속하는지 판별합니다.
2. Link Prediction: Node간에 연결이 되었는지 Prediction 합니다.

이번 .ipynb에서는 GNN과 GAT를 사용하였을 경우에 Node Classification의 성능이 어떠한 변화를 보이는지 확인합니다.

### Import Library

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 6)
pd.set_option("display.max_rows", 6)
np.random.seed(2)

### Obtain the dataset

**Cora Dataset**

2708개의 논문에 대한 내용이 담겨있는 예제입니다. 해당 Dataset의 Node는 각 논문의 ID이고, Edge는 서로 인용하였는가 입니다. 해당 Dataset에서 Label은 논문의 종류로서 총 7 가지 입니다. 기존에 사용한 Notation으로서는 다음과 같이 적을수 있습니다.

- $N$: Number of nodes (2708)
- $F$: Dimension of node (1433)

In [2]:
zip_file = keras.utils.get_file(
    fname="cora.tgz",
    origin="https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz",
    extract=True,
)

data_dir = os.path.join(os.path.dirname(zip_file), "cora")

citations = pd.read_csv(
    os.path.join(data_dir, "cora.cites"),
    sep="\t",
    header=None,
    names=["target", "source"],
)

papers = pd.read_csv(
    os.path.join(data_dir, "cora.content"),
    sep="\t",
    header=None,
    names=["paper_id"] + [f"term_{idx}" for idx in range(1433)] + ["subject"],
)

class_values = sorted(papers["subject"].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
citations["source"] = citations["source"].apply(lambda name: paper_idx[name])
citations["target"] = citations["target"].apply(lambda name: paper_idx[name])
papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])

Citation 정보를 담고 있는 Dataset 입니다. 서로 이어진 Edge를 의미합니다.

In [3]:
print(citations)

      target  source
0          0      21
1          0     905
2          0     906
...      ...     ...
5426    1874    2586
5427    1876    1874
5428    1897    2707

[5429 rows x 2 columns]


Paper의 정보를 가지고 있습니다. 각 Paper의 Id와 특성 그리고 Label정보를 가지고 있습니다.

In [4]:
print(papers)

      paper_id  term_0  term_1  ...  term_1431  term_1432  subject
0          462       0       0  ...          0          0        2
1         1911       0       0  ...          0          0        5
2         2002       0       0  ...          0          0        4
...        ...     ...     ...  ...        ...        ...      ...
2705      2372       0       0  ...          0          0        1
2706       955       0       0  ...          0          0        0
2707       376       0       0  ...          0          0        2

[2708 rows x 1435 columns]


### Split Dataset
Dataset을 Split합니다. 50% 기준으로 잘라서 Train Dataset과 Test Dataset을 구축 합니다.

In [5]:
# Obtain random indices
random_indices = np.random.permutation(range(papers.shape[0]))

# 50/50 split
train_data = papers.iloc[random_indices[: len(random_indices) // 2]]
test_data = papers.iloc[random_indices[len(random_indices) // 2 :]]

### Prepare Dataset

Train Dataset과 Test Dataset을 Numpy -> Tensor로서 변형합니다.

In [6]:
# Obtain paper indices which will be used to gather node states
# from the graph later on when training the model
train_indices = train_data["paper_id"].to_numpy()
test_indices = test_data["paper_id"].to_numpy()

# Obtain ground truth labels corresponding to each paper_id
train_labels = train_data["subject"].to_numpy()
test_labels = test_data["subject"].to_numpy()

# Define graph, namely an edge tensor and a node feature tensor
edges = tf.convert_to_tensor(citations[["target", "source"]])
node_states = tf.convert_to_tensor(papers.sort_values("paper_id").iloc[:, 1:-1])

# Print shapes of the graph
print("Edges shape:\t\t", edges.shape)
print("Node features shape:", node_states.shape)

Edges shape:		 (5429, 2)
Node features shape: (2708, 1433)


2023-04-13 20:42:18.258857: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Tensorflow Function

현재 Code를 보기 전에 앞서서 사용하는 Tensorflow의 함수를 이해하고 가야한다.

**Function.1 : tf.gather**

인덱스의 단일 축 텐서를 전달하는 방법 이다. 현재 사용중인 Edge의 Dataset은 아래와 같이 생겼다.
```code
[[   0,   21],
[   0,  905],
[   0,  906],
...,
[1874, 2586],
[1876, 1874],
[1897, 2707]]
```

따라서, 해당 결과는 입력되는 Tensor에서 해당되는 Index 2개 (ex) [0, 21])를 concat하는 역할을 한다.

Example

In [7]:
params = tf.constant(['p0', 'p1', 'p2', 'p3', 'p4', 'p5'])
indices = [2, 0, 2, 5]
tf.gather(params, indices).numpy()

array([b'p2', b'p0', b'p2', b'p5'], dtype=object)

**Function.2: tf.math.unsorted_segment_sum**

Segmentation을 진하는 연산이다. 아래 코드를 보면 이해가 빠르다. 같은 Segment끼리 합해주는 과정을 가진다.

Case 1: c의 0, 2번째가 같은 Cluster이고, 1번째가 다른 Cluster 이다.

In [5]:
c = [[1,2,3,4], [5,6,7,8], [4,3,2,1]]
tf.math.unsorted_segment_sum(c, [0, 1, 0], num_segments=2).numpy()

array([[5, 5, 5, 5],
       [5, 6, 7, 8]], dtype=int32)

Case 2: c의 0, 1번째가 같은 Cluster이고, 2번째가 다른 Cluster 이다.

In [3]:
c = [[1,2,3,4], [5,6,7,8], [4,3,2,1]]
tf.math.unsorted_segment_sum(c, [0, 0, 1], num_segments=2).numpy()

array([[ 6,  8, 10, 12],
       [ 4,  3,  2,  1]], dtype=int32)

Case 3: c의 0, 1번째가 같은 Cluster이고, 2번째가 다른 Cluster 이다.

In [4]:
c = [[1,2,3,4], [5,6,7,8], [4,3,2,1]]
tf.math.unsorted_segment_sum(c, [1, 1, 0], num_segments=2).numpy()

array([[ 4,  3,  2,  1],
       [ 6,  8, 10, 12]], dtype=int32)

위의 Case를 살표보면, <code>tf.math.unsorted_segment_sum</code>은 같은 Cluster값을 더해주며, 순서는 Cluter의 Label 이다.

<span style="color:red">**사용시 주의 사항**</span>

위의 Code중에서 <code>tf.repeat(attention_scores_sum, tf.math.bincount(tf.cast(edges[:, 0], "int32")))</code>부분 때문에 Edge의 값은 항상 Sorting되어 있어나 같은 값들이 뭉쳐있어야 한다. 즉, Edge가 무조건 Sorting되어 있다고 가정하고 사용하는 Code 이다. (Graph쪽은 Dataset을 항상 이렇게 구축하는지는 잘 모르겠습니다.)

In [8]:
a_s = [[1],
       [4],
       [7]]

e = [0, 0, 0, 1, 1, 2, 2, 2]

a_s_t = tf.constant(a_s)
e_t = tf.constant(e)
print(tf.repeat(a_s_t, tf.math.bincount(e_t)))

a_s = [[1],
       [4],
       [7]]

e = [0, 0, 1, 0, 1, 2, 2, 2]

a_s_t = tf.constant(a_s)
e_t = tf.constant(e)
print(tf.repeat(a_s_t, tf.math.bincount(e_t)))

tf.Tensor([1 1 1 4 4 7 7 7], shape=(8,), dtype=int32)
tf.Tensor([1 1 1 4 4 7 7 7], shape=(8,), dtype=int32)


### Attention Mechanism

![png](./tmp/img/2.png)

아래 Code는 Graph Attention Network에서 Attention Mechanism에 대한 Layer입니다. 수식을 대입해서 확인하기 정의해야하는 Notation과 해당 부분은 아래와 같습니다. 또한, Dimension이 헛갈리기 때문에 예시로 실제 값을 적어두겠습니다.

**Notation**
- <code>input_shape[0][-1]</code>: Dimension of Feature ($F$) (1433)
- <code>self.units</code>: Dimension of hidden Layer ($F^{'}$) (4)
- <code>self.kernel</code>: Weight (Trainable Parameter) ($W \in \mathbb{R}^{F \times F^{'}}$) (1433 x 4)
- <code>self.kernel_attention</code>: For Attention( Trainable Parameter) ($a \in \mathbb{R}^{2F^{'}}$) (8 x 1)

해당 Model에서 Input으로 들어오는 정보는 아래와 같습니다.

**Inputs**
- <code>node_states</code>: Input ($\vec{h} = \{\vec{h_1}, \vec{h_2}, \ldots, \vec{h_N} \} \in \mathbb{R}^{N \times F}$) (2708 x 4)
- <code>edges</code>: For Attention (Node간에 이여져 있는지 확인하기 위하여 사용) (5429, 2)
    - 0: i-th Node
    - 1: j-th Node
    
아래 구현된 Code와 논문에서 제시한 방법을 대입하면 아래와 같습니다.

**Attention Mechanism**
- #1: $W \vec{h}$ // Input을 Weight를 곱하여 Latent Representation으로 보낸다.
    - <code>node_states_transformed = tf.matmul(node_states, self.kernel)</code> (2708 x 4)

- #2: $(W\vec{h_i}, W\vec{h_j})$ // Concat을 한다. 여기서 뒤따라 오는 수식 때문에, 모든 조합에 대하여 계산할 필요는 없다. 연결된 조합에 대하여서만 Concat 필요하며, 이는 edges에 정보가 담겨있다. Output은 조합된 개수(Edge shape[0] x attemtion dimension)으로서 나온다.
    - <code>tf.gather(node_states_transformed, edges)</code> (5429, 2, 4)
    - <code>node_states_expanded = tf.reshape(node_states_expanded, (tf.shape(edges)[0], -1))</code> (5429, 8)
    
- #3: $LeakyReLU(a(W\vec{h_i}, W\vec{h_j}))$ // Attention Score를 구한다.
    - <code>attention_scores = tf.nn.leaky_relu(tf.matmul(node_states_expanded, self.kernel_attention))</code> (5429, 1)
    - <code>attention_scores = tf.squeeze(attention_scores, -1)</code> (5429,)
    
- #4: $exp(LeakyReLU(e_{ik}))$ // Exponential을 취한다. (Softmax 계산을 위해)
    - <code>attention_scores = tf.math.exp(tf.clip_by_value(attention_scores, -2, 2))</code> (5429, )
- #5: $\sum_{k \in N_i} exp(LeakyReLU(e_{ik}))$ // 연결된 조합에 대한 Attention Score의 합을 구한다. 해당 과정에 대해서는 아래 2 Step으로서 이루워진다.
    - <code>attention_scores_sum = tf.math.unsorted_segment_sum(data=attention_scores, segment_ids=edges[:, 0],
            num_segments=tf.reduce_max(edges[:, 0]) + 1,)</code>: (edges[:, 0]의 개수-1898, ) // 나를 기준으로 이어진 Node는 같은 Segments로서 지정하여 Summation 취한다. 따라서 해당 Output은 기준이 되는 Edge의 Unique한 개수가 된다.
    - <code>attention_scores_sum = tf.repeat(attention_scores_sum, 
    tf.math.bincount(tf.cast(edges[:, 0], "int32")))</code>: (5429,) // Repeat하여 개수를 증가시킨다. Attention Distribution을 계산하기 위해서 이다.

- #6: $\alpha_{ij} = \frac{exp( LeakyReLU(e_{ij}))}{\sum_{k \in N_i} exp(LeakyReLU(e_{ik}))}$ // Attention Distribution을 구한다.
    - <code>attention_scores_norm = attention_scores / attention_scores_sum</code>

- #7: $\vec{h^{'}_i} = \sigma(\sum_{j \in N_j} \alpha_{ij} W \vec{h_j})$ // Graph Attention 값을 구한다. Output은 해당 노드와 주변 노드간의 Weight및 Attention을 곱한 값으로서 표현된다. 
    - <code>node_states_neighbors = tf.gather(node_states_transformed, edges[:, 1])</code> (5429, 4) // Output으로 뱉고자 하는 Node ($\vec{h_i}^{'}$)와 연결되 Node의 특성 ($W\vec{h_j}$)을 가져온다.
    - <code>out = tf.math.unsorted_segment_sum(
            data=node_states_neighbors * attention_scores_norm[:, tf.newaxis],
            segment_ids=edges[:, 0],
            num_segments=tf.shape(node_states)[0],)</code>  (2708, 4) // Output ($\vec{h_i}^{'}$)은 이어져 있는 주변 Node의 정보 ($W\vec{h_j}$)와 Attention Score(해당 Node i에 대하여 주변에 연결되어 있는 Node들 중에 특정 Node j가 영향을 미치는 정도)의 곱으로서 나타낸다.

In [10]:
class GraphAttention(layers.Layer):
    def __init__(
        self,
        units,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.units = units
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)

    def build(self, input_shape):

        self.kernel = self.add_weight(
            shape=(input_shape[0][-1], self.units),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel",
        )
        self.kernel_attention = self.add_weight(
            shape=(self.units * 2, 1),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel_attention",
        )
        self.built = True

    def call(self, inputs):
        node_states, edges = inputs

        # Linearly transform node states
        node_states_transformed = tf.matmul(node_states, self.kernel)

        # (1) Compute pair-wise attention scores
        node_states_expanded = tf.gather(node_states_transformed, edges)
        node_states_expanded = tf.reshape(
            node_states_expanded, (tf.shape(edges)[0], -1)
        )
        attention_scores = tf.nn.leaky_relu(
            tf.matmul(node_states_expanded, self.kernel_attention)
        )
        attention_scores = tf.squeeze(attention_scores, -1)

        # (2) Normalize attention scores
        attention_scores = tf.math.exp(tf.clip_by_value(attention_scores, -2, 2))
        attention_scores_sum = tf.math.unsorted_segment_sum(
            data=attention_scores,
            segment_ids=edges[:, 0],
            num_segments=tf.reduce_max(edges[:, 0]) + 1,
        )
        attention_scores_sum = tf.repeat(
            attention_scores_sum, tf.math.bincount(tf.cast(edges[:, 0], "int32"))
        )
        attention_scores_norm = attention_scores / attention_scores_sum

        # (3) Gather node states of neighbors, apply attention scores and aggregate
        node_states_neighbors = tf.gather(node_states_transformed, edges[:, 1])
        out = tf.math.unsorted_segment_sum(
            data=node_states_neighbors * attention_scores_norm[:, tf.newaxis],
            segment_ids=edges[:, 0],
            num_segments=tf.shape(node_states)[0],
        )
        return out

### (Multi-head) graph attention layer

해당 부분은 간단한 부분입니다.

![png](./tmp/img/3.png)

위의 Figure와 같이 여러 Attention Layer에서 나온 Output을 Concat후 Average하여 사용합니다.
$$\vec{h^{'}_i} = \sigma(\frac{1}{K} \sum_{k=1}^K \sum_{j \in N_j} \alpha_{ij}^k W^k \vec{h_j})$$

In [11]:
class MultiHeadGraphAttention(layers.Layer):
    def __init__(self, units, num_heads=8, merge_type="concat", **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.merge_type = merge_type
        self.attention_layers = [GraphAttention(units) for _ in range(num_heads)]

    def call(self, inputs):
        atom_features, pair_indices = inputs

        # Obtain outputs from each attention head
        outputs = [
            attention_layer([atom_features, pair_indices])
            for attention_layer in self.attention_layers
        ]
        # Concatenate or average the node states from each head
        if self.merge_type == "concat":
            outputs = tf.concat(outputs, axis=-1)
        else:
            outputs = tf.reduce_mean(tf.stack(outputs, axis=-1), axis=-1)
        # Activate and return node states
        return tf.nn.relu(outputs)

### Graph Attention Network

해당 모델의의 Architecture는 총 3개의 Layer로서 구성되어있습니다.
- 1st Layer: <code>self.preprocess = layers.Dense(hidden_units * num_heads, activation="relu")</code> // 간단한 Dense Layer로서 처음에 많은 Feature를 적은수의 Feature로서 변환한다.
- 2nd Layer: <code>self.attention_layers = [MultiHeadGraphAttention(hidden_units, num_heads) for _ in range(num_layers)]</code>: num_layers만큼 MultiHead Graph Attention Network를 적용한다. 몇개 self attention을 사용하는 지는 num_heads로서 지정한다.
- 3rd Layer: <code>self.output_layer = layers.Dense(output_dim)</code>: 간단하게 Dense Layer로서 Node Classification을 실시한다.

해당 Model Architecture에서 주의하여서 봐야하는 점은 아래와 같다.

**Appendix #1. No Edge Information**
```python
for attention_layer in self.attention_layers:
    x = attention_layer([x, edges]) + x
```

위의 Code를 보게 되면, Input에서 받게 되는 정보의 Edge Dataset은 자기 자신은 이어져 있다고 (Self-Connection)생각하지 않습니다. 하지만, 이러한 자신의 정보를 사용하지 않고 주변 정보만을 사용하게 된다면, Performance는 매우 낮은 것을 확인할 수 있습니다. (대략 30% 감소) 따라서 위와 같이 자기 자신의 정보를 그대로 사용하는 과정이 필요합니다.

해당 과정을 수식으로 쓰면 다음과 같습니다.
$$\vec{h^{'}_i} = \sigma(\frac{1}{K} \sum_{k=1}^K \sum_{j \in N_j} \alpha_{ij}^k W^k \vec{h_j})$$
$$\rightarrow \vec{h^{'}_i} = \sigma(\frac{1}{K} \sum_{k=1}^K \sum_{j \in N_j, j \neq i} \alpha_{ij}^k W^k \vec{h_j}) + \vec{h_i}$$

위의 수식을 사용하게 되면, 자기 자신의 정보를 중요하게 생각하면서, 다른 주변 연결된 Node 중에서 중요한 것을 강조하여 사용하는 형태의 수식이 됩니다.

**Appendix #2. Indexing**
```python
def train_step(self, data):
    indices, labels = data

    with tf.GradientTape() as tape:
        # Forward pass
        outputs = self([self.node_states, self.edges])
        # Compute loss
        loss = self.compiled_loss(labels, tf.gather(outputs, indices))
        
    # Compute gradients
    grads = tape.gradient(loss, self.trainable_weights)
    # Apply gradients (update weights)
    optimizer.apply_gradients(zip(grads, self.trainable_weights))
    # Update metric(s)
    self.compiled_metrics.update_state(labels, tf.gather(outputs, indices))

    return {m.name: m.result() for m in self.metrics}
```

현재 Dataset을 구축할 때, Node를 기준으로 나누었다. 또한, Node에 대한 연결정보를 사용하기 위하여 아래와 같은 Dataset으로서 Graph를 구성하였다.

- <code>node_states</code>: (2708, 1433)
- <code>edges</code>: (5429, 2)

구축한 Dataset에서 GAT의 Output은 (2708, num_classes)로서 모든 Node에 대한 정보가 들어가있다. 때문에 위와 같이 <code>tf.gather(outputs, indices)</code>로서 train sample에 대한 probability (1304, num_classes)와 train label을 비교하면서 학습을 진행하여야 한다.

위에서 train_step 안의 코드는 형식화 되있는 부분이며, 이해가 되지 않으면 <a href="https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit?hl=ko">Tensorflow Model-fit</a>를 참조하자.

In [12]:
class GraphAttentionNetwork(keras.Model):
    def __init__(
        self,
        node_states,
        edges,
        hidden_units,
        num_heads,
        num_layers,
        output_dim,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.node_states = node_states
        self.edges = edges
        self.preprocess = layers.Dense(hidden_units * num_heads, activation="relu")
        self.attention_layers = [
            MultiHeadGraphAttention(hidden_units, num_heads) for _ in range(num_layers)
        ]
        self.output_layer = layers.Dense(output_dim)

    def call(self, inputs):
        node_states, edges = inputs
        x = self.preprocess(node_states)
        for attention_layer in self.attention_layers:
            x = attention_layer([x, edges]) + x
            
        outputs = self.output_layer(x)
        return outputs

    def train_step(self, data):
        indices, labels = data

        with tf.GradientTape() as tape:
            # Forward pass
            outputs = self([self.node_states, self.edges])
            # Compute loss
            loss = self.compiled_loss(labels, tf.gather(outputs, indices))
        # Compute gradients
        grads = tape.gradient(loss, self.trainable_weights)
        # Apply gradients (update weights)
        optimizer.apply_gradients(zip(grads, self.trainable_weights))
        # Update metric(s)
        self.compiled_metrics.update_state(labels, tf.gather(outputs, indices))

        return {m.name: m.result() for m in self.metrics}

    def predict_step(self, data):
        indices = data
        # Forward pass
        outputs = self([self.node_states, self.edges])
        # Compute probabilities
        return tf.nn.softmax(tf.gather(outputs, indices))

    def test_step(self, data):
        indices, labels = data
        # Forward pass
        outputs = self([self.node_states, self.edges])
        # Compute loss
        loss = self.compiled_loss(labels, tf.gather(outputs, indices))
        # Update metric(s)
        self.compiled_metrics.update_state(labels, tf.gather(outputs, indices))

        return {m.name: m.result() for m in self.metrics}

### Train and evaluate

In [13]:
# Define hyper-parameters
HIDDEN_UNITS = 100
NUM_HEADS = 8
NUM_LAYERS = 3
OUTPUT_DIM = len(class_values)

NUM_EPOCHS = 100
BATCH_SIZE = 256
VALIDATION_SPLIT = 0.1
LEARNING_RATE = 3e-1
MOMENTUM = 0.9

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.SGD(LEARNING_RATE, momentum=MOMENTUM)
accuracy_fn = keras.metrics.SparseCategoricalAccuracy(name="acc")
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_acc", min_delta=1e-5, patience=5, restore_best_weights=True
)

# Build model
gat_model = GraphAttentionNetwork(
    node_states, edges, HIDDEN_UNITS, NUM_HEADS, NUM_LAYERS, OUTPUT_DIM
)

# Compile model
gat_model.compile(loss=loss_fn, optimizer=optimizer, metrics=[accuracy_fn])

gat_model.fit(
    x=train_indices,
    y=train_labels,
    validation_split=VALIDATION_SPLIT,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[early_stopping],
    verbose=2,
)

_, test_accuracy = gat_model.evaluate(x=test_indices, y=test_labels, verbose=0)

print("--" * 38 + f"\nTest Accuracy {test_accuracy*100:.1f}%")

Epoch 1/100
5/5 - 55s - loss: 1.9140 - acc: 0.2718 - val_loss: 1.7719 - val_acc: 0.4265 - 55s/epoch - 11s/step
Epoch 2/100
5/5 - 11s - loss: 1.4025 - acc: 0.5181 - val_loss: 1.1628 - val_acc: 0.6618 - 11s/epoch - 2s/step
Epoch 3/100
5/5 - 9s - loss: 0.8099 - acc: 0.7668 - val_loss: 0.8047 - val_acc: 0.8088 - 9s/epoch - 2s/step
Epoch 4/100
5/5 - 6s - loss: 0.4824 - acc: 0.8645 - val_loss: 0.8099 - val_acc: 0.7794 - 6s/epoch - 1s/step
Epoch 5/100
5/5 - 6s - loss: 0.2931 - acc: 0.9113 - val_loss: 0.8186 - val_acc: 0.8235 - 6s/epoch - 1s/step
Epoch 6/100
5/5 - 11s - loss: 0.1972 - acc: 0.9483 - val_loss: 0.7035 - val_acc: 0.8088 - 11s/epoch - 2s/step
Epoch 7/100
5/5 - 11s - loss: 0.1096 - acc: 0.9819 - val_loss: 0.7641 - val_acc: 0.8088 - 11s/epoch - 2s/step
Epoch 8/100
5/5 - 10s - loss: 0.0686 - acc: 0.9918 - val_loss: 0.7421 - val_acc: 0.8162 - 10s/epoch - 2s/step
Epoch 9/100
5/5 - 9s - loss: 0.0480 - acc: 0.9951 - val_loss: 0.7110 - val_acc: 0.8529 - 9s/epoch - 2s/step
Epoch 10/100
5/5 